# Job-Hunt NLP Demo

Which demo will also be useful in doing some quick NLP work to see how my résumé's word distribution matches that from job descriptions.

There's a wonderful project out there, [MyBinder](https://mybinder.org), which allows you to interactively run a Jupyter notebook completely online. It's nice to have when you'd like to play with code and see better the outputs that come from running that code. I've had some problems with images going down, but I'm going to work to keep this one up.

The link to the online, interactive notebook - the binder - is at the badge you see right here

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/bballdave025/job-app-word-freq/main?labpath=A_02nd_NLPPresentationJobHunt_DemoWordFreq.ipynb)

<hr/>

## We are calling this version 0.1.002

It's for the FamilySearch CJKV jobs applied for in August 2023.

<hr/>

## Link to setup from the Conda Prompt

The instructions for setting up the conda environment from Windows is in [my first version, here on GitHub](https://github.com/bballdave025/job-app-word-freq/blob/main/A_v01_NLP_Presentation_Job_Hunt_NLP_Useful_Demo_Word_Freq.ipynb). Soon, I will figure out how to make the [MyBinder](https://mybinder.org) server <strike>[MyBinder server]()</strike> for that first version persistent, and you can look at all the setup stuff there. [![Binder](./badge_logo_dwb_v_0-1-001_merged_small.png)](https://mybinder.org/v2/gh/bballdave025/job-app-word-freq/original-timed-freq?labpath=A_v01_NLP_Presentation_Job_Hunt_NLP_Useful_Demo_Word_Freq.ipynb)

## The Second Iteration

### Several FamilySearch Résumés

My friend at the [FamilySearch Library](https://www.familysearch.org/en/library/) let me know about a few job availabilities. These are all with a group - of which he and I are part - of missionaries and volunteers who have been working on [CJKV (Chinese, Japanese, Korean, Vietnamese)-character](https://en.wikipedia.org/wiki/CJKV_characters) handwriting and block-print recognition. I already put in the applications with résumés, but all résumés are pretty simple. I'm going to see how the different job descriptions compare to the résumés as regards the word-frequency distribution. 

I'm going to add some improvements to my first, time-limited version. These include the better-presentation output of the word and frequency arrays. I would also like to add something that removes small words that serve a more grammatical function; in the [NLTK book](https://www.nltk.org/book/) (Officially: BIRD, Steven; KLEIN, Ewan; and LOPER, Edward, Natural Language Processing with Python – Analyzing Text with the Natural Language Toolkit, retrieved 2023-08-07, https://web.archive.org/web/20230721043038/https://www.nltk.org/book/)
Steven Bird, Ewan Klein, and Edward Loper), these are called stopwords (cf. Chapter 2). Perhaps I'll put in some n-gram comparison, but what I'd really like to do are space-separated bigrams - now I've looked it up, and the official term is Orthogonal Sparse Bigrams (OSBs), cf [this paper](https://web.archive.org/web/20230807215040/https://www.siefkes.net/papers/winnow-spam.pdf). The paper is also in this directory as `siefkes_winnow_osb.pdf`.

## Texts

The text of my résumé for these jobs is in the local file,

```
res_CJKV.txt
```

the job descriptions for the jobs are in local files as well, specifically,

```
desc_CJKV_dev3.txt
desc_CJKV_dev4.txt
desc_CJKV_dev5.txt
desc_CJKV_devInTest3.txt
```

In [2]:
application_text_filenames = \
  ["res_CJKV.txt",
  ]

In [3]:
job_description_text_filenames = \
  ["desc_CJKV_dev5.txt",
   "desc_CJKV_dev4.txt",
   "desc_CJKV_dev3.txt",
   "desc_CJKV_devInTest3.txt",
  ]

# The "dev5" is the nicest job - and it's with Java, which I know best.

`######################################################`

The job description page looks to contain something like `JavaScript`, `ajax`, etc.

Rather than writing in a webscraper or looking through the code and finding what gets pulled from the database, I'm just going to copy/paste the text into the text files.

In [4]:
##  Code to get current timestamp, if needed.
##+ Meant to be run once, then commented out.
# #######################
# # No need to run again
# #####
# !powershell -c (Get-Date -UFormat "%s_%Y%m%dT%H%M%S%Z00") -replace '[.][0-9]*_', '_'

In [5]:
local_job_desc_filenames = job_description_text_filenames
local_job_appl_filenames = application_text_filenames

import pprint

pprint.pprint(local_job_desc_filenames)
print()
pprint.pprint(local_job_appl_filenames)

['desc_CJKV_dev5.txt',
 'desc_CJKV_dev4.txt',
 'desc_CJKV_dev3.txt',
 'desc_CJKV_devInTest3.txt']

['res_CJKV.txt']


Output was

```
['desc_CJKV_dev5.txt',
 'desc_CJKV_dev4.txt',
 'desc_CJKV_dev3.txt',
 'desc_CJKV_devInTest3.txt']

['res_CJKV.txt']
```

at `1691423942_20230807T155902-0600`

In [11]:
# read in texts to original strings

def read_in_texts(local_desc_fnames, local_appl_fnames,
                  do_combine_desc_files = False,
                  do_combine_appl_files = True
                 ):
    for_pairwise_desc_texts = []
    for_pairwise_appl_texts = []
    
    complete_description_text = ""
    complete_application_text = ""
    
    if do_combine_desc_files:
        complete_description_text = " "
    ##endof:  if do_combine-desc_files
    
    if do_combine_appl_files:
        complete_application_text = " "
    ##endof:  if do_combine_appl_files
    
    for this_description_filename in local_job_desc_filenames:
        with open(this_description_filename, 'r', encoding='utf-8') as dfh:
            this_desc_file_content_str = dfh.read()
            if do_combine_desc_files:
                complete_description_text += " " + this_desc_file_content_str
            else:
                this_desc_in_array_str = " " + this_desc_file_content_str + " "
                for_pairwise_desc_texts.append(this_desc_in_array_str)
            ##endof:  if/else do_combine_desc_files
        ##endof:  with open ... dfh
    ##endof:  for this_description_filename in local_job_desc_filenames
    
    for this_application_filename in local_job_appl_filenames:
        with open(this_application_filename, 'r', encoding='utf-8') as afh:
            this_appl_file_content_str = afh.read()
            if do_combine_appl_files:
                complete_application_text += " " + this_appl_file_content_str
            else:
                this_appl_in_array_str = " " + this_appl_file_content_str + " "
                for_pairwise_appl_texts.append(this_appl_in_array_str)
            ##endof:  if/else do_combine_appl_files
        ##endof:  with open ... afh
    ##endof:  for this_application_filename in local_job_appl_filenames
    
    complete_description_text += " "
    complete_application_text += " "
    
    if do_combine_desc_files:
        for_pairwise_desc_texts = [complete_description_text]
    ##endof:  if do_combine-desc_files
    
    if do_combine_appl_files:
        for_pairwise_appl_texts = [complete_application_text]
    ##endof:  if do_combine_appl_files
    
    return for_pairwise_desc_texts, for_pairwise_appl_texts
    
##endof:  read_in_texts(<params>)

In [12]:
# # read in texts to original strings
# #  The original
# complete_description_text = " "
# complete_application_text = " "
# 
# for this_description_filename in local_job_desc_filenames:
#     with open(this_description_filename, 'r', encoding='utf-8') as dfh:
#         complete_description_text += dfh.read()
#     ##endof:  with open ... dfh
# ##endof:  for this_description_filename in local_job_desc_filenames
# 
# for this_application_filename in local_job_app_filenames:
#     with open(this_application_filename, 'r', encoding='utf-8') as afh:
#         complete_application_text += afh.read()
#     ##endof:  with open ... afh
# ##endof:  for this_application_filename in local_job_app_filenames
# 
# complete_description_text += " "
# complete_application_text += " "

#### This next, make_it_one_line_single_spaced function will be very useful as we go forward

In [38]:
import string
import re

def make_it_one_line_single_spaced(input_str):
    processing_str = input_str
    
    processing_str = ' '.join(processing_str.split())
    processing_str = processing_str.replace("\t", " ")
    processing_str = processing_str.replace("\n", " ")
    processing_str = re.sub(r"(^|[^ ])[ ][ ]+($|[^ ])",
                            r"\g<1> \g<2>",
                            processing_str,
                            flags=re.IGNORECASE
                           )
    
    return processing_str
##endof:  make_it_one_line_single_spaced(input_str)

### The actual reading in of the texts

In [13]:
complete_description_text, complete_application_text = \
                    read_in_texts(local_job_desc_filenames,
                                  local_job_appl_filenames)

### Code for cleaning text

We will iterate a bit, so as not to have to write a text normalizer for the whole world. Rather than putting together regexes to test for things like which contractions are there and which other things might need changing (especially things like dashes), I'm doing simple regexes. Q&R

In [46]:
import re
import string

#from bs4 import BeautifulSoup
#from bs4 import UnicodeDammit

def clean_text_string_quickly(input_str):
    processing_str = input_str
    
    # ## one line, single-spaced
    # processing_str = ' '.join(processing_str.split())
    # processing_str = processing_str.replace("\t", " ")
    # processing_str = processing_str.replace("\n", " ")
    # processing_str = re.sub(r"(^|[^ ])[ ][ ]+($|[^ ])",
    #                         r"\g<1> \g<2>",
    #                         processing_str,
    #                         flags=re.IGNORECASE
    #                       )
    
    ## one line, single-spaced
    processing_str = make_it_one_line_single_spaced(processing_str)
    
    
    ## get rid of outside-ascii (or control character)
    processing_str = re.sub(r"[^\u0020-\u007E]",
                            " ",
                            processing_str,
                            flags=re.IGNORECASE
                           )
    
    ## my stuff
    processing_str = re.sub(r"[ ][|]+[ ]",
                            " ",
                            processing_str,
                            flags=re.IGNORECASE
                           )
    processing_str = processing_str.replace(r"&", "and")
    processing_str = processing_str.replace(r"U.S.", "U S ")
    
    ## get rid of punctuation
    processing_str = re.sub(r"(([^0-9 ])[.,!?:\"']([) ]|$))",
                            r"\g<2>\g<3>",
                            processing_str,
                            flags=re.IGNORECASE
                           )
    processing_str = re.sub(r"(([0-9 ])[.,!?:\"']([ ]|$))",
                            r"\g<2>\g<3>",
                            processing_str,
                            flags=re.IGNORECASE
                           )
    # parentheses
    processing_str = processing_str.replace(r"(", " ")
    processing_str = processing_str.replace(r")", " ")
    # dashes
    processing_str = re.sub(r"[ ][-]+[ ]",
                            " ",
                            processing_str,
                            flags=re.IGNORECASE
                           )
    
    ##  lowercase - to skip until a few iterations through
    ##+ cleaning the text
    processing_str = processing_str.casefold()
    
    ## fixes found by iterating this cleaning function
    processing_str = re.sub(r"[ ][/][ ]",
                            " ",
                            processing_str,
                            flags=re.IGNORECASE
                           )
    
    ## What's found in the documents
    # My inspection
    processing_str = processing_str.replace(r"s.r", "s r")
    processing_str = processing_str.replace(r"c++/perl", "c++ perl")
    
    # From the automated looking, below
    processing_str = processing_str.replace(
                                 r"monitors/equipment", 
                                  "monitors equipment"
    )
    processing_str = processing_str.replace(
                                 r"product/engineering", 
                                  "product engineering"
    )
    processing_str = processing_str.replace(
                                 r"engineering/troubleshooting", 
                                  "engineering troubleshooting"
    )
    processing_str = processing_str.replace(
                                 r"engineering/programming", 
                                  "engineering programming"
    )
    processing_str = processing_str.replace(
                         r"analytical/diagnostic/troubleshooting", 
                          "analytical diagnostic troubleshooting"
    )
    processing_str = processing_str.replace(
                                 r"integration/continuous", 
                                  "integration continuous"
    )
    
    processing_str = processing_str.replace(r"net/powershell", 
                                                "net powershell")
    processing_str = processing_str.replace(r"c/c", "c c")
    
    # KEEP THESE 3 EXAMPLES IN THE CODE FOR COPY/PASTE, WHATEVER
    # processing_str = processing_str.replace(r"notice/more", 
    #                                               "notice more")
    # processing_str = processing_str.replace(r"s.r", "s r")
    # processing_str = processing_str.replace(
    #                              r"monitors/equipment", 
    #                               "monitors equipment"
    # )
    
    # ##spacing fix at the end
    # processing_str = re.sub(r"(^|[^ ])[ ][ ]+($|[^ ])",
    #                         r"\g<1> \g<2>",
    #                         processing_str,
    #                         flags=re.IGNORECASE
    #                        )
    
    ## spacing fix at the end
    processing_str = make_it_one_line_single_spaced(processing_str)
    
    ## Let's give it back
    return processing_str

##endof:  clean_text_string_quickly(input_str)

In [47]:
import re

def remove_stopwords(input_str):
    ##  From https://www.nltk.org/book/ch02.html
    ##+ > [Stopwords are] high-frequency words like the, to and also that we 
    ##+ > sometimes want to filter out of a document before further processing. 
    ##+ > Stopwords usually have little lexical content, and their presence in 
    ##+ > a text fails to distinguish it from other texts.
    
    processing_str = input_str
    
    stopwords_to_remove = [
'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours',
'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers',
'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves',
'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are',
'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does',
'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until',
'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into',
'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down',
'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here',
'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',
'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so',
'than', 'too', 'very', 'can', 'will', 'just', 'should', 'now'
]
    
    # No attempt to optimize code, here. Q&R
    for my_stopword in stopwords_to_remove:
        #processing_str = processing_str.replace(my_stopword, " ")
        word_with_boundaries = r"\b" + my_stopword + r"\b"
        processing_str = re.sub(word_with_boundaries, " ", 
                                processing_str, 
                                flags=re.IGNORECASE)
    ##endof:  for my_stopword in stopwords_to_remove
    
    return processing_str
##endof:  remove_stopwords(input_str)

In [48]:
# We'll usually keep these two.
do_look_at_description_text = True
do_look_at_application_text = True

#  this one can go (False) if you don't want the big strings
#+ i.e. you don't want the complete file contents
do_print_the_big_strings = True

In [49]:
if do_look_at_description_text:
    test1 = []
    for desc_text_str in complete_description_text:
        test1.append(clean_text_string_quickly(desc_text_str))
        if do_print_the_big_strings:
            import pprint
            pprint.pprint(test1)

['familysearch software dev engineer 5 us-based remote optional lehi ut united '
 'states united states remote to meet the needs of the church we seek to build '
 'teams that represent the diverse perspectives broad life experiences and '
 'backgrounds of our global church membership with that in mind we encourage '
 'all qualified applicants to apply job description the software dev engineer '
 '5 is a master professional full-stack role that requires vast knowledge of '
 'industry practices and organizational policies the role requires good '
 'judgment in selecting methods and techniques for designing defining '
 'developing testing purchasing implementing and analyzing technical products '
 'and systems of diverse scope this position recommends system purchases for '
 'high value projects this position fully participates as a technical leader '
 'and influences numerous engineering teams and leads in areas such as '
 'software development project planning activities work break-down

In [50]:
if do_look_at_application_text:
    test2 = []
    for appl_text_str in complete_application_text:
        test2.append(clean_text_string_quickly(appl_text_str))
        if do_print_the_big_strings:
            import pprint
            pprint.pprint(test2)

['david black github.com/bballdave025 linkedin.com/in/thedavidwblack +1 '
 '801-376-4520 thedavidwblack@gmail.com stackexchange.com/users/8693193 75 '
 'peregrine ln #8 bountiful ut 84010 usa motivation this position as '
 'familysearch software dev engineer 5 357360 aligns with my passions '
 'interests and beliefs combining my lifelong passion for family history and '
 "science with programming and technical expertise excites me the workplace's "
 'focus on faith and its mandate to integrate it into work inspire me previous '
 'work at familysearch involved preserving and sharing family stories using '
 'machine learning coding and genealogical records i eagerly anticipate '
 'collaborating with diverse and faithful team members guided by the savior in '
 'helping people through the technical aspect of family history highlights 5+ '
 'years machine-learning research automated speech and handwriting recognition '
 '11+ years technical software development and computational research 2 

In [51]:
## Without stopwords
test1 = [make_it_one_line_single_spaced(
                                remove_stopwords(their_text)
                                       ) for their_text in test1
        ]

if do_print_the_big_strings:
    import pprint
    pprint.pprint(test1)

['familysearch software dev engineer 5 us-based remote optional lehi ut united '
 'states united states remote meet needs church seek build teams represent '
 'diverse perspectives broad life experiences backgrounds global church '
 'membership mind encourage qualified applicants apply job description '
 'software dev engineer 5 master professional full-stack role requires vast '
 'knowledge industry practices organizational policies role requires good '
 'judgment selecting methods techniques designing defining developing testing '
 'purchasing implementing analyzing technical products systems diverse scope '
 'position recommends system purchases high value projects position fully '
 'participates technical leader influences numerous engineering teams leads '
 'areas software development project planning activities work break- tasks '
 'engineering peer reviews etc position responsible technical leadership '
 'architecture design implementation software quality -going engineering '
 

In [52]:
test2 = [make_it_one_line_single_spaced(
                                remove_stopwords(their_text)
                                       ) for their_text in test2
        ]

if do_print_the_big_strings:
    import pprint
    pprint.pprint(test2)

['david black github.com/bballdave025 linkedin.com/ /thedavidwblack +1 '
 '801-376-4520 thedavidwblack@gmail.com stackexchange.com/users/8693193 75 '
 'peregrine ln #8 bountiful ut 84010 usa motivation position familysearch '
 'software dev engineer 5 357360 aligns passions interests beliefs combining '
 'lifelong passion family history science programming technical expertise '
 "excites workplace's focus faith mandate integrate work inspire previous work "
 'familysearch involved preserving sharing family stories using machine '
 'learning coding genealogical records eagerly anticipate collaborating '
 'diverse faithful team members guided savior helping people technical aspect '
 'family history highlights 5+ years machine-learning research automated '
 'speech handwriting recognition 11+ years technical software development '
 'computational research 2 years post-secondary teaching experience u s patent '
 '11017778 automated speech recognition technology issued may 25 2021 aws '
 '

### For these next few cells, we are finding things to search and replace

In [53]:
# looking at contractions
if do_look_at_description_text:
    for their_text in test1:
        print()
        print(re.findall(r"\b('[\w']+\b|[\w']+'[\w']+|[\w']+')\b",
                         their_text)
             )


["organization's", "bachelor's", "master's"]

["bachelor's"]

["bachelor's"]

["bachelor's"]


First run-through had

```
["organization's", "bachelor's", "master's"]

["bachelor's"]

["bachelor's"]

["bachelor's"]
```

In [54]:
if do_look_at_application_text:
    for my_text in test2:
        print()
        print(re.findall(r"\b('[\w']+\b|[\w']+'[\w']+|[\w']+')\b", 
                         my_text)
             )


["workplace's", "wife's", "nist's", "container's", "mission's"]


First run-through had

```
["workplace's", "wife's", "nist's", "container's", "mission's"]
```

In [55]:
# looking at all slashes
if do_look_at_description_text:
    for their_text in test1:
        print()
        print(re.findall(r"\b[\w/]+/[\w/]+\b", 
                         their_text)
             )


[]

[]

[]

[]


First run-through had

```
['monitors/equipment']

['product/engineering', 'engineering/troubleshooting', 'monitors/equipment']

['monitors/equipment']

['engineering/programming', 'analytical/diagnostic/troubleshooting', 'monitors/equipment', 'integration/continuous']
```

In [56]:
if do_look_at_application_text:
    for my_text in test2:
        print()
        print(re.findall(r"\b[\w./]+/[\w/]+\b",
                         my_text)
             )


['github.com/bballdave025', 'stackexchange.com/users/8693193']


First run-through had

```
['github.com/bballdave025', 'stackexchange.com/users/8693193', 'net/powershell', 'c/c']
```

## Word Frequency Counts

I want to use an `OrderedDict`, rather than mess with sorting the contents of a `dict`.

In [57]:
#import sys
from collections import OrderedDict

def get_sorted_word_counts(*cleaned_strings):
                           #,
                           #do_output_sorted_file=False,
                           #sorted_filename=\
                           #    "sorted_words_from_strings.txt"):
    '''
    @return  OrderedDict
    '''
    
    EXIT_NOWORDSWEREFOUND = -1
    
    work_with_str = combine_strings(cleaned_strings)
    
    list_of_words_in_str = work_with_str.split()
    
    if len(list_of_words_in_str) <= 0:
        print("No words were found.", file=sys.stdout)
        print("The program will exit.", file=sys.stdout)
        #sys.exit(EXIT_NOWORDSWEREFOUND)
        return EXIT_NOWORDSWEREFOUND
    ##endof:  if len(list_of_words_in_str) <= 0
    
    word_count_ordered_dict = OrderedDict()
    
    for this_word in list_of_words_in_str:
        if this_word in word_count_ordered_dict:
            word_count_ordered_dict[this_word] += 1
        else:
            word_count_ordered_dict[this_word] = 1
        ##endof:  if/else this_word in list_of_words_in_str
    ##endof:  for this_word in list_of_words_in_str
    
    ## DWB note ##
    ##  At this point, the OrderedDict is sorted by the
    ##+ order in which keys were inserted, not by their
    ##+ count.
    
    for key, _ in \
          sorted(word_count_ordered_dict.items(),
                 key=lambda word_and_count: word_and_count[1],
                 reverse=True):
        word_count_ordered_dict.move_to_end(key)
    ##endof:  for myword, _ ...
    
    return word_count_ordered_dict
    
##endof:  get_sorted_word_counts(*cleaned_strings)

def combine_strings(tuple_of_strings):
                    #, 
                    #do_output_raw_file=False,
                    #raw_filename='raw_words_from_strings.txt'):
    '''
    @return  string
    '''
    
    returned_str = " "
    
    for this_str in tuple_of_strings:
        returned_str += this_str + " "
    ##endof:  for this_str in tuple_of_string
    
    ## one line, single-spaced
    returned_str = ' '.join(returned_str.split())
    returned_str = returned_str.replace("\t", " ")
    returned_str = returned_str.replace("\n", " ")
    returned_str = re.sub(r"([^ ])[ ][ ]+($|[^ ])",
                            r"\g<1> \g<2>",
                            returned_str,
                            flags=re.IGNORECASE
                           )
    
    return returned_str
##endof:  combine_strings(tuple_of_strings)

# @TODO: add a sort-by-word as well as sort-by-count flag
# @TODO:  also, print out the pre-sorted and sorted files
#       + with word lists, frequency, and in-order-of-
#       + highest-count stuff

**Only do the two below if you want a big preview! What I'm saying is, "The two cells below will give you long outputs if uncommented."**

In [58]:
complete_description_text

[" FamilySearch Software Dev Engineer 5 (US-based, Remote Optional)\nLehi, UT, United States\nUnited States\n(Remote)\nTo meet the needs of the Church, we seek to build teams that represent the diverse perspectives, broad life experiences and backgrounds of our global Church membership. With that in mind, we encourage all qualified applicants to apply.\n\nJOB DESCRIPTION\nThe Software Dev Engineer 5 is a master, professional full-stack role that requires vast knowledge of industry practices, and organizational policies. The role requires good judgment in selecting methods and techniques for designing, defining, developing, testing, purchasing, implementing, and analyzing technical products and systems of diverse scope. This position recommends system purchases for high value projects. This position fully participates as a technical leader and influences numerous engineering teams, and leads in areas such as software development, project planning activities, work break-down of tasks, en

In [59]:
complete_application_text

["  David BLACK\n\n\n\n\n\n\n\n\ngithub.com/bballdave025 | linkedin.com/in/thedavidwblack\n| +1 801-376-4520  |\nthedavidwblack@gmail.com\nstackexchange.com/users/8693193 \n| 75 Peregrine Ln. #8  Bountiful, UT  84010   USA\n\n\nMOTIVATION\n\nThis position, as FamilySearch Software Dev Engineer 5 (357360), aligns with my passions, interests, & beliefs. Combining my lifelong passion for Family History & Science with programming and technical expertise excites me. The workplace's focus on faith and its mandate to integrate it into work inspire me. Previous work at FamilySearch involved preserving & sharing family stories using machine learning, coding, & genealogical records. I eagerly anticipate collaborating with diverse & faithful team members, guided by the Savior in helping people through the technical aspect of family history.\n\nHIGHLIGHTS\n\n  »\t5+ years machine-learning research - automated speech & handwriting recognition\n  »\t11+ years technical software development & computa

In [66]:
## Cleaning and Counting ##

description_strings_pre = \
  [clean_text_string_quickly(this_desc_text_str) 
           for this_desc_text_str in complete_description_text]
application_strings_pre = \
  [clean_text_string_quickly(this_appl_text_str)
           for this_appl_text_str in complete_application_text]

description_strings = \
  [make_it_one_line_single_spaced(remove_stopwords(this_desc)) 
           for this_desc in description_strings_pre]
application_strings = \
  [make_it_one_line_single_spaced(remove_stopwords(this_appl)) 
           for this_appl in application_strings_pre]

description_word_counts = \
  [get_sorted_word_counts(description_str) \
              for description_str in description_strings]
application_word_counts = \
  [get_sorted_word_counts(application_str) \
              for application_str in application_strings]

**Once again, the four cells below will give you long outputs if uncommented.**

In [67]:
description_strings

["familysearch software dev engineer 5 us-based remote optional lehi ut united states united states remote meet needs church seek build teams represent diverse perspectives broad life experiences backgrounds global church membership mind encourage qualified applicants apply job description software dev engineer 5 master professional full-stack role requires vast knowledge industry practices organizational policies role requires good judgment selecting methods techniques designing defining developing testing purchasing implementing analyzing technical products systems diverse scope position recommends system purchases high value projects position fully participates technical leader influences numerous engineering teams leads areas software development project planning activities work break- tasks engineering peer reviews etc position responsible technical leadership architecture design implementation software quality -going engineering support organization's technology applications indi

In [77]:
description_word_counts

[OrderedDict([('software', 10),
              ('work', 8),
              ('team', 7),
              ('church', 6),
              ('technical', 6),
              ('engineering', 6),
              ('experience', 6),
              ('development', 5),
              ('professional', 4),
              ('role', 4),
              ('product', 4),
              ('programming', 4),
              ('seek', 3),
              ('teams', 3),
              ('job', 3),
              ('industry', 3),
              ('practices', 3),
              ('position', 3),
              ('areas', 3),
              ('design', 3),
              ('support', 3),
              ('technology', 3),
              ('provide', 3),
              ('gospel', 3),
              ('develop', 3),
              ('processes', 3),
              ('technologies', 3),
              ('computer', 3),
              ('skills', 3),
              ('ability', 3),
              ('dev', 2),
              ('engineer', 2),
              ('5', 2),
    

In [69]:
application_strings

["david black github.com/bballdave025 linkedin.com/ /thedavidwblack +1 801-376-4520 thedavidwblack@gmail.com stackexchange.com/users/8693193 75 peregrine ln #8 bountiful ut 84010 usa motivation position familysearch software dev engineer 5 357360 aligns passions interests beliefs combining lifelong passion family history science programming technical expertise excites workplace's focus faith mandate integrate work inspire previous work familysearch involved preserving sharing family stories using machine learning coding genealogical records eagerly anticipate collaborating diverse faithful team members guided savior helping people technical aspect family history highlights 5+ years machine-learning research automated speech handwriting recognition 11+ years technical software development computational research 2 years post-secondary teaching experience u s patent 11017778 automated speech recognition technology issued may 25 2021 aws machine learning specialist certified august 3 2023 

In [70]:
application_word_counts

[OrderedDict([('python', 10),
              ('software', 8),
              ('java', 6),
              ('familysearch', 5),
              ('family', 5),
              ('programming', 5),
              ('technical', 5),
              ('learning', 5),
              ('team', 5),
              ('research', 5),
              ('data', 5),
              ('science', 4),
              ('years', 4),
              ('automated', 4),
              ('aws', 4),
              ('bash', 4),
              ('c++', 4),
              ('tools', 4),
              ('transcription', 4),
              ('ut', 3),
              ('work', 3),
              ('machine', 3),
              ('records', 3),
              ('speech', 3),
              ('experience', 3),
              ('s', 3),
              ('technology', 3),
              ('2023', 3),
              ('languages', 3),
              ('perl', 3),
              ('french', 3),
              ('linux', 3),
              ('normalization', 3),
              ('scoring

The next code will take care of the issue I described thusly:
> It's annoying me not to have a nice, aligned output for these 2d lists - basically, they're tables. I need to bring in some previous code that takes care of getting stuff printed nice. That will be after the Q&R.

In [71]:
import string
from io import StringIO
import re

def print_2d_list_columns_aligned(list_2d_to_print,
                                  joining_delimiter = ",  ",
                                  do_output_as_str_and_print = False,
                                  do_see_the_guts=False):
    ##  The  do_see_the_guts boolean is partly for debugging,
    ##+ partly for remembering and teaching how the process
    ##+ works.
    
    ## Make all elements strings - so we can use len()
    list_2d_all_strings = \
      [[str(item) for item in row] for row in list_2d_to_print]
    
    if do_see_the_guts:
        print()
        print("  list_2d_all_strings:")
        print(list_2d_all_strings)
        print()
    ##endof:  if do_see_the_guts
    
    #  We want to find the max string length for each column
    #+ We can basically transpose the 2d_list to get the
    #+ content of each column
    list_of_column_elems_as_tuples = \
                 [column for column in zip(*list_2d_all_strings)]
    
    if do_see_the_guts:
        print()
        print("  list_of_column_elems_as_tuples:")
        print(list_of_column_elems_as_tuples)
        print()
    ##endof:  if do_see_the_guts
    
    ## find the max string length for each tuple (each column)
    list_of_max_str_len_by_column = \
      [max([len(strng) for strng in tpl]) 
        for tpl in list_of_column_elems_as_tuples]
    # -v- gives array with elements being each longest string
    #[max([strng for strng in tpl], key=len) for tpl in list_of_column_elems_as_tuples]
    # -v- 2d array with strings
    #[[strng for strng in tpl] for tpl in list_of_column_elems_as_tuples]
    
    if do_see_the_guts:
        print()
        print("  list_of_max_str_len_by_column:")
        print(list_of_max_str_len_by_column)
        print()
    ##endof:  if do_see_the_guts
    
    # output_as_str_not_list = False
    
    # Create a formatter for each row
    
    #if not output_as_str_not_list:
    joining_delimiter = "," + joining_delimiter
    
    fmt_str = \
      joining_delimiter.join('{{:{}}}'.format(max_len) 
                               for max_len in list_of_max_str_len_by_column)
    #if not output_as_str_not_list:
    fmt_str = "[" + fmt_str + "],"
    
    if do_see_the_guts:
        print()
        print("  fmt_str:")
        print(fmt_str)
        print()
    ##endof:  if do_see_the_guts
    
    # Get a string for each row, formatted correctly
    list_of_formatted_row_strings = \
      [fmt_str.format(*row) for row in list_2d_all_strings]
    
    if do_see_the_guts:
        print()
        print("  list_of_formatted_row_strings:")
        print(list_of_formatted_row_strings)
        print()
    ##endof:  if do_see_the_guts
    
    # if output_as_str_not_list:
    #     aligned_table_to_return = '\n'.join(list_of_formatted_row_strings)
    #     
    #     print(aligned_table_to_return)
    #     return aligned_table_to_return
    # ##endof:  if output_as_str_not_list
    
    s = StringIO()
    print(*list_of_formatted_row_strings, file=s)
    output_table_raw = s.getvalue()
    
    output_table_raw = re.sub(r"^\[", r"[[ ", output_table_raw)
    output_table_raw = re.sub(r"\],$", r"]]", output_table_raw)
    #output_table_raw = re.sub(r"([^ ])\],", r"\g<1> ],", output_table_raw)
    output_table_raw = re.sub(r"\],", r" ],", output_table_raw)
    output_table_raw = re.sub(r",,", r" ,", output_table_raw)
    output_table_raw = re.sub(r"]]", r" ]]", output_table_raw)
    
    aligned_table_to_return = output_table_raw.replace(r"], [", "],\n [ ")
    
    print(aligned_table_to_return)
    
    if do_output_as_str_and_print:
        return aligned_table_to_return
    ##endof:  if do_output_as_str_and_print
    
##endof:  print_2d_list_colunns_aligned(<params>)

In [72]:
# ##  Uncomment if you want to see the guts of a small example
# ##+ that's been hacked into working all right
# print_2d_list_columns_aligned([['hey', 7], ['work', 3], ['stupid', 2]], do_see_the_guts=True)

In [82]:
## Only change this boolean to True if you want to see a lot of output. ##
do_print_long_full_version = True

if do_print_long_full_version:
    #import pprint
    
    dashes="------------------------------------------------------------"
    short_dashes="-----"
    
    this_d_str_counter = -1 # quick hack for zero-indexed
    
    for d_word_count_dict in description_word_counts:
        this_d_str_counter += 1
        
        #  I haven't yet written this for several resumes, so
        #+ it just compares each description with the first
        #+ (or, likely, combined) thing in the resume stuff.
        this_a_str_index = 0
        a_word_count_dict = application_word_counts[this_a_str_index]
        
        print()
        print()
        print(dashes)
        print(f" For the job description with index, {this_d_str_counter}")
        print( " (meaning it's from the file:")
        print(f"   {local_job_desc_filenames[this_d_str_counter]}),")
        print(short_dashes)
        this_d_wdct_items_list = list(d_word_count_dict.items())
        this_d_wdct_2d_list = [list (ele) for ele in this_d_wdct_items_list]
        #pprint.pprint(this_d_wdct_2d_list)
        print_2d_list_columns_aligned(this_d_wdct_2d_list)
        print(dashes)
        print()
        print()
        print(dashes)
        print( " For the job application material (résumé, cover letter, etc.)")
        print(f" with index, {this_a_str_index}")
        print( "(meaning it's from the file:")
        print(f"   {local_job_appl_filenames[this_a_str_index]}),")
        print(short_dashes)
        this_a_wdct_items_list = list(a_word_count_dict.items())
        this_a_wdct_2d_list = [list (ele) for ele in this_a_wdct_items_list]
        #pprint.pprint(this_a_wdct_2d_list)
        print_2d_list_columns_aligned(this_a_wdct_2d_list)
        print(dashes)
        print()
        print()
        print(dashes)
        print(dashes)
        print()
        print()
    ##endof:  for d_word_count_dict in description_word_counts
##endof:  if do_print_long_full_version



------------------------------------------------------------
 For the job description with index, 0
 (meaning it's from the file:
   desc_CJKV_dev5.txt),
-----
[[ software         ,  10 ],
 [ work             ,  8  ],
 [ team             ,  7  ],
 [ church           ,  6  ],
 [ technical        ,  6  ],
 [ engineering      ,  6  ],
 [ experience       ,  6  ],
 [ development      ,  5  ],
 [ professional     ,  4  ],
 [ role             ,  4  ],
 [ product          ,  4  ],
 [ programming      ,  4  ],
 [ seek             ,  3  ],
 [ teams            ,  3  ],
 [ job              ,  3  ],
 [ industry         ,  3  ],
 [ practices        ,  3  ],
 [ position         ,  3  ],
 [ areas            ,  3  ],
 [ design           ,  3  ],
 [ support          ,  3  ],
 [ technology       ,  3  ],
 [ provide          ,  3  ],
 [ gospel           ,  3  ],
 [ develop          ,  3  ],
 [ processes        ,  3  ],
 [ technologies     ,  3  ],
 [ computer         ,  3  ],
 [ skills           ,  3  

In [79]:
desc_counter = -1 # hack for zero-indexing
appl_index   =  0 #  combining application files
                  #+ (actually, here, there's only one file)

# These start out impossible, so we'll know if we use it without changing
this_d_tie_dict_index = -1
this_d_tie_value = -1
this_a_tie_dict_index = -1
this_a_tie_value = -1

for this_d_word_count in description_word_counts:
    desc_counter += 1
    
    #  I haven't yet written this for several resumes, so
    #+ it just compares each description with the first
    #+ (or, likely, combined) thing in the resume stuff.
    this_a_word_count = application_word_counts[appl_index]
    
    
    ##  We will do the following for headers
    table_version_desc = [["desc_word", "desc_cnt", "desc_rank"],]
    table_version_appl = [["appl_word", "appl_cnt", "appl_rank"],]
    table_version_both = [["rank", "desc_word", "desc_cnt", 
                           "appl_word", "appl_cnt"],
                         ]
    
    description_items = list(this_d_word_count.items())
    application_items = list(this_a_word_count.items())
    
    n_words_description = len(description_items)
    n_words_application = len(application_items)
    
    print()
    print("NOTE THAT THESE ARE THE NUMBERS OF DISTINCT WORDS")
    print()
    print(f" For the job description with index, {desc_counter}")
    print( " (meaning it's from the file:")
    print(f"   {local_job_desc_filenames[desc_counter]}),")
    print("AND")
    print( " For the job application material (résumé, cover letter, etc.)")
    print(f" with index, {this_a_str_index}")
    print( "(meaning it's from the file:")
    print(f"   {local_job_appl_filenames[this_a_str_index]}),")
    print()
    print(f"n_words_description = {str(n_words_description)}")
    print(f"n_words_application = {str(n_words_application)}")
    print()
    print()
    
##endof:  for this_d_word_count in description_word_counts


NOTE THAT THESE ARE THE NUMBERS OF DISTINCT WORDS
n_words_description = 329
n_words_application = 480




DESCRIPTION
[('software', 10),
 ('work', 8),
 ('team', 7),
 ('church', 6),
 ('technical', 6),
 ('engineering', 6),
 ('experience', 6),
 ('development', 5),
 ('professional', 4),
 ('role', 4),
 ('product', 4),
 ('programming', 4),
 ('seek', 3),
 ('teams', 3),
 ('job', 3),
 ('industry', 3),
 ('practices', 3),
 ('position', 3),
 ('areas', 3),
 ('design', 3),
 ('support', 3),
 ('technology', 3),
 ('provide', 3),
 ('gospel', 3),
 ('develop', 3),
 ('processes', 3),
 ('technologies', 3),
 ('computer', 3),
 ('skills', 3),
 ('ability', 3),
 ('dev', 2),
 ('engineer', 2),
 ('5', 2),
 ('remote', 2),
 ('united', 2),
 ('states', 2),
 ('needs', 2),
 ('build', 2),
 ('diverse', 2),
 ('global', 2),
 ('full-stack', 2),
 ('requires', 2),
 ('knowledge', 2),
 ('designing', 2),
 ('defining', 2),
 ('implementing', 2),
 ('products', 2),
 ('systems', 2),
 ('system', 2),
 ('leader', 2),
 ('project', 2),
 ('

In [ ]:
import sys

for this_idx in range(max(len(description_items),
                          len(application_items)
                         )
                      ):
    this_rank = this_idx + 1
    
    if this_idx < len(description_items) - 1:
        try:
            this_description_word  = description_items[this_idx+1][0]
        except IndexError as ie:
            print("OTHER ERROR desc word!", file=sys.stdout)
            print(str(ie), file=sys.stdout)
            print(f"this_idx: {str(this_idx)}", file=sys.stdout)
            print(f"this_rank: {str(this_rank)}", file=sys.stdout)
            print(f"len(description_items): {str(len(description_items))}", file=sys.stdout)
            end_of_data_bool_try = ( this_idx <= len(description_items) )
            print(f"end_of_data_bool_try: {str(end_of_data_bool_try)}", file=sys.stdout)
        finally:
            pass
        ##endof:  try/catch/finally
        
        try:
            this_description_count = description_items[this_idx+1][1]
        except IndexError as ie:
            print("OTHER ERROR desc count!", file=sys.stdout)
            print(str(ie), file=sys.stdout)
            print(f"this_idx: {str(this_idx)}", file=sys.stdout)
            print(f"this_rank: {str(this_rank)}", file=sys.stdout)
            print(f"len(description_items): {str(len(description_items))}", file=sys.stdout)
            end_of_data_bool_try = ( this_idx <= len(description_items) )
            print(f"end_of_data_bool_try: {str(end_of_data_bool_try)}", file=sys.stdout)
        finally:
            pass
        ##endof:  try/catch/finally
        
        this_description_rank  = this_rank
    else:
        this_description_word  = "-- N/A --"
        this_description_count = "-- N/A --"
        this_description_rank  = "-- N/A --"
    ##endof:  if/else this_idx < len(description_items)
    
    if this_idx < len(application_items) - 1:
        try:
            this_application_word  = application_items[this_idx+1][0]
        except IndexError as ie:
            print("OTHER ERROR appl word!", file=sys.stdout)
            print(str(ie), file=sys.stdout)
            print(f"this_idx: {str(this_idx)}", file=sys.stdout)
            print(f"this_rank: {str(this_rank)}", file=sys.stdout)
            print(f"len(application_items): {str(len(application_items))}", file=sys.stdout)
            end_of_data_bool_try = ( this_idx <= len(application_items) )
            print(f"end_of_data_bool_try: {str(end_of_data_bool_try)}", file=sys.stdout)
        finally:
            pass
        ##endof:  try/catch/finally
        
        try:
            this_application_count = application_items[this_idx+1][1]
        except IndexError as ie:
            print("OTHER ERROR appl count!", file=sys.stdout)
            print(str(ie), file=sys.stdout)
            print(f"this_idx: {str(this_idx)}", file=sys.stdout)
            print(f"this_rank: {str(this_rank)}", file=sys.stdout)
            print(f"len(application_items): {str(len(application_items))}", file=sys.stdout)
            end_of_data_bool_try = ( this_idx <= len(application_items) )
            print(f"end_of_data_bool_try: {str(end_of_data_bool_try)}", file=sys.stdout)
        finally:
            pass
        ##endof:  try/catch/finally
        
        this_application_rank  = this_rank
    else:
        this_application_word  = "-- N/A --"
        this_application_count = "-- N/A --"
        this_application_rank  = "-- N/A --"
    ##endof:  if/else this_idx < len(application_items)
    
    if this_description_word != "-- N/A --":
        try:
            table_version_desc.append([this_description_word, 
                                       this_description_count, 
                                       this_rank]
                                     )
        except IndexError as ie:
            print("ERROR desc!", file=sys.stdout)
            print(str(ie), file=sys.stdout)
            print(f"this_idx: {str(this_idx)}", file=sys.stdout)
            print(f"this_rank: {str(this_rank)}", file=sys.stdout)
            print(f"this_description_word: {str(this_description_word)}", file=sys.stdout)
            print(f"this_description_count: {str(this_description_count)}", file=sys.stdout)
        finally:
            pass
        ##endof:  try/except/finally
        
    ##endof:  if this_description_word != "-- N/A --"
    
    if this_application_word != "-- N/A --":
        try:
            table_version_appl.append([this_application_word, 
                                       this_application_count, 
                                       this_rank
                                      ]
                                     )
        except IndexError as ie:
            print("ERROR appl!", file=sys.stdout)
            print(str(ie), file=sys.stdout)
            print(f"this_idx: {this_idx}", file=sys.stdout)
            print(f"this_rank: {str(this_rank)}", file=sys.stdout)
            print(f"this_application_word: {str(this_application_word)}", file=sys.stdout)
            print(f"this_application_count: {str(this_application_count)}", file=sys.stdout)
        finally:
            pass
        ##endof:  try/except/finally
        
    ##endof:  if this_application_word != "-- N/A --"
    
    try:
        table_version_both.append([this_rank, 
                                   this_description_word, 
                                   this_description_count,
                                   this_application_word, 
                                   this_application_count
                                  ]
                                 )
    except IndexError as ie:
        print("ERROR both!", file=sys.stdout)
        print(str(ie), file=sys.stdout)
        print(f"this_idx: {this_idx}", file=sys.stdout)
        print(f"this_rank: {str(this_rank)}", file=sys.stdout)
        print(f"this_description_word: {str(this_description_word)}", file=sys.stdout)
        print(f"this_description_count: {str(this_description_count)}", file=sys.stdout)
        print(f"this_application_word: {str(this_application_word)}", file=sys.stdout)
        print(f"this_application_count: {str(this_application_count)}", file=sys.stdout)
    finally:
        pass
    ##endof:  try/except/finally
        
##endof:  for this_idx in ..

In [ ]:
###
##  Set up the display the first n_lines_to_display 
##+ of the tables, nicely

#### This next one is the one you might change
n_lines_to_display_orig = 25

n_header_lines = 1
n_lines_to_display = n_lines_to_display_orig
n_lines_to_display_desc = n_lines_to_display_orig
n_lines_to_display_appl = n_lines_to_display_orig

do_cut_down_desc = ( n_lines_to_display_orig >
                             len(table_version_desc) 
                   )
if do_cut_down_desc:
    n_lines_to_display_desc = len(table_version_desc)
##endof:  if do_cut_down_desc

do_cut_down_appl = ( n_lines_to_display_orig >
                             len(table_version_appl) 
                   )
if do_cut_down_appl:
    n_lines_to_display_appl = len(table_version_appl)
##endof:  if do_cut_down_appl

# get headers
display_table_desc = [table_version_desc[0]]
display_table_appl = [table_version_appl[0]]
display_table_both = [table_version_both[0]]

if ( len(table_version_desc) - n_header_lines < n_lines_to_display or
     len(table_version_appl) - n_header_lines < n_lines_to_display
   ):
    n_lines_to_display = min(len(table_version_desc) - n_header_lines,
                             len(table_version_appl) - n_header_lines)
##endof:  if <n_lines_conditions>


for table_idx in range(n_header_lines, 
                       n_lines_to_display_orig + n_header_lines):
    if table_idx - n_header_lines < n_lines_to_display_desc:
        try:
            display_table_desc.append(table_version_desc[table_idx])
        except IndexError as ie:
            print("ERROR display_table_desc!", file=sys.stdout)
            print(str(ie), file=sys.stdout)
            print(f"table_idx: {table_idx}", file=sys.stdout)
            print(f"n_lines_to_display_desc: {n_lines_to_display_desc}", file=sys.stdout)
            print(f"len(table_version_desc): {len(table_version_desc)}", file=sys.stdout)
        except Error as e:
            print("DIFFERENT ERROR display_table_desc!", file=sys.stdout)
            print(str(e), file=sys.stdout)
            print(f"table_idx: {table_idx}", file=sys.stdout)
            print(f"n_lines_to_display_desc: {n_lines_to_display_desc}", file=sys.stdout)
            print(f"len(table_version_desc): {len(table_version_desc)}", file=sys.stdout)
        finally:
            pass
        ##endof:  try/catch/finally
    ##endof:  if table_idx - n_header_lines < n_lines_to_display_desc
    
    if table_idx - n_header_lines < n_lines_to_display_appl:
        try:
            display_table_appl.append(table_version_appl[table_idx])
        except IndexError as ie:
            print("ERROR display_table_appl!", file=sys.stdout)
            print(str(ie), file=sys.stdout)
            print(f"table_idx: {table_idx}", file=sys.stdout)
            print(f"n_lines_to_display_appl: {n_lines_to_display_appl}", file=sys.stdout)
            print(f"len(table_version_appl): {len(table_version_appl)}", file=sys.stdout)
        except Error as e:
            print("DIFFERENT ERROR display_table_appl!", file=sys.stdout)
            print(str(e), file=sys.stdout)
            print(f"table_idx: {table_idx}", file=sys.stdout)
            print(f"n_lines_to_display_appl: {n_lines_to_display_appl}", file=sys.stdout)
            print(f"len(table_version_appl): {len(table_version_appl)}", file=sys.stdout)
        finally:
            pass
        ##endof:  try/catch/finally
    ##endof:  if table_idx - n_header_lines < n_lines_to_display_appl
    
    try:
        display_table_both.append(table_version_both[table_idx])
    except IndexError as ie:
        print("ERROR display_table_both!", file=sys.stdout)
        print(str(ie), file=sys.stdout)
        print(f"table_idx: {table_idx}", file=sys.stdout)
        print(f"len(table_version_both): {len(table_version_both)}", file=sys.stdout)
    finally:
        pass
    ##endof:  try/catch/finally
##endof:  for idx in range(<n_lines stuff>)

In [ ]:
print(f"len(table_version_desc): {len(table_version_desc)}")
print(f"len(display_table_desc): {len(display_table_desc)}")
print()
print(f"len(table_version_appl): {len(table_version_appl)}")
print(f"len(display_table_appl): {len(display_table_appl)}")
print()
print(f"len(table_version_both): {len(table_version_both)}")
print(f"len(display_table_both): {len(display_table_both)}")

In [ ]:
#import pprint

long_dashes = "-----------------------------------------------"
short_dashes = "-----"

print()
print(long_dashes)
print("JOB DESCRIPTION (TOP 25)")
print(short_dashes)
#pprint.pprint(display_table_desc)
print_2d_list_columns_aligned(display_table_desc)
print()
print(long_dashes)
print()
print()
print(long_dashes)
print("JOB APPLICATION STUFF - RéSUMé AND COVER LETTER (TOP 25)")
print(short_dashes)
#pprint.pprint(display_table_appl)
print_2d_list_columns_aligned(display_table_appl)
print()
print(long_dashes)
print()
print()
print(long_dashes)
print("COMPARISON OF DESCRIPTION AND APPLICATION (TOP 25)")
print(short_dashes)
#pprint.pprint(display_table_both)
print_2d_list_columns_aligned(display_table_both)
print()
print(long_dashes)

## Time for histograms (or whatever the discretized version is)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Next line only for Jupyter notebook.
%matplotlib inline

def get_histo_from_freq_dict(word_count_ordered_dict,
                             n_top_words = 25,
                             do_show_word_and_count_lists=False,
                             axx=None
                            ):
    '''
    @return  an axis from matplotlab (with the object - histogram - in it)
    '''
    
    if axx is None:
        fig = plt.figure(figsize=(10, 3))
        axx = fig.add_subplot(111)
    
    counts_pre = list(word_count_ordered_dict.values())
    words_pre  = list(word_count_ordered_dict.keys())
    
    counts = counts_pre[:n_top_words]
    words  = words_pre[:n_top_words]
    
    ## making sure things were working
    if do_show_word_and_count_lists:
        print(f"counts: {counts}")
        print(f"words:  {words}")
    ##endof:  if do_show_word_and_count_lists
    
    x_words_coords = np.arange(len(words))
    axx.bar(x_words_coords, counts, align='center')
    
    axx.set_xticks(x_words_coords)
    axx.set_xticklabels(words, rotation=45, ha='right')
    
##endof:  get_histo_from_freq_dict

In [ ]:
get_histo_from_freq_dict(description_word_counts, do_show_word_and_count_lists=True)
plt.show()

In [ ]:
get_histo_from_freq_dict(application_word_counts)
plt.show()

## Output for Description and Application:

### &lt;FILL THIS IN&gt;

### Done

In [ ]:
# #######################
# # No need to run again
# #####
# !powershell -c (Get-Date -UFormat "%s_%Y%m%dT%H%M%S%Z00") -replace '[.][0-9]*_', '_'

The output when I actually did this was

```
<Here is where the output will go>
```

The output histograms, in an image.

<br/>
<div>
  <img src="first_QandR_word_frequency_plots.jpg"
       alt="The first pair of histograms - one for the job description, one for the job application - with word frequencies"
       width="100%">
</div>
<br/>

Here is some idea of how they match. I hope it makes some sense. Darker green means an exact match; thinner dark green means a match with words that don't add much meaning; lighter green means it's a close match. 

<br/>
<div>
  <img src="first_QandR_word_frequency_plots_w_link_lines.jpg"
       alt="Word matches for the first pair of histograms."
       width="100%">
</div>
<br/>

## Future Steps

- Look at ranking, counts, percentage, etc. for FamilySearch's (job description's) top 25 words as found in my (job application's) word counts, then vice-versa. 
- Get rid of words that are necessary for grammar, but which don't matter too much in determining whether the two documents match up. (Found term on 2023-08-17. It's "stopwords".)